In [1]:
import pandas as pd
import os
from astropy.utils.data import download_file
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import shutil


# Already arranged the way I want it, no more processing needed
duration_data = pd.read_csv('DataFrames/duration_data_Fermi.csv', index_col=0)
# Need to process
fluence_data = pd.read_table('summary/Best_fit_fluence_Fermi.dat', sep = '|', skipinitialspace = True, skiprows = 4, usecols = [1,2,3,4,5,6])

In [2]:
# Assigning best fit fluences and saving to a new file
fluence_data = fluence_data.iloc[:,[0,1,2,3,4,5]]
fluence_data.columns = ['name', 'best_fit', 'flnc_plaw', 'flnc_comp', 'flnc_band', 'flnc_sbpl']
fluence_data = fluence_data.set_index('name')
fluence_data.loc[:,'best_fit'] = fluence_data.loc[:,'best_fit'].apply(lambda x: str(x).strip())
for x in fluence_data.index:
    try:
        fluence_data.loc[x,'best_fit'] = str(fluence_data.loc[x,fluence_data.loc[x,'best_fit']]) # Setting as string to show scientific notation
    except:
        fluence_data.loc[x,'best_fit'] = float('NaN')
fluence_data = fluence_data.iloc[:,[0]]
fluence_data.columns = ['fluence']
fluence_data.to_csv('DataFrames/fluence_data_Fermi.csv')

In [3]:
## Downloading each burst's bcat .fit file -- have to brute-force to find the version number
def download_files():
    grb_names = duration_data.index
    for grb in grb_names:
        for i in range(5):
            url = f'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/bursts/20{grb[3:5]}/bn{grb[3:]}/current/glg_bcat_all_bn{grb[3:]}_v0{i}.fit'
            try:
                tmp_path = download_file(url)
                #print(f'file found at {url}')
                batlc_path = f"Fermi/bn{grb[3:]}.fit"
                shutil.move(tmp_path, batlc_path)
            except:
                pass
    return

#download_files()